In [1]:
import _pickle as pickle
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import copy
import glob
import gc
import ipdb
import time

In [2]:
pd.set_option('precision',2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('max_rows',200)

In [3]:
full_df = []

exp_list = glob.glob('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_*sapbert.2000-NN_DataFrame.p')

print(exp_list)

for filename in tqdm(exp_list):
    file = open(filename,'rb')
    full_df.append(pickle.load(file))
    file.close()

['/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_sapbert.2000-NN_DataFrame.p']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [06:55<00:00, 415.16s/it]


In [4]:
umls2020AB_df = full_df[0]

In [ ]:
for df in full_df[1:]:
    nn_columns = df.filter(regex='.*NN.*').columns
    for col in nn_columns:
        umls2020AB_df[col] = df[col]

In [5]:
del full_df

In [6]:
umls2020AB_df.columns

Index(['0', 'strings', 'auis', '2020AA_synonyms', 'synonym_strings',
       'num_syms', 'sapbert_2000-NN_strings', 'sapbert_2000-NN_auis',
       'sapbert_2000-NN_dist', 'sapbert_2000-NN_recall'],
      dtype='object')

In [ ]:
for col in umls2020AB_df.filter(regex='lexlm.*').columns:
    umls2020AB_df.drop(col,axis=1)

In [ ]:
nearest_neighbors_auis = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/lex_lm_2000-NN.p','rb'))
nearest_neighbors_dist = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/lex_lm_2000-NN_dist.p','rb'))
nearest_neighbors_auis = [auis for auis in nearest_neighbors_auis]

original_umls_2020, new_umls_2020 = pickle.load(open('aui_string_map_UMLS2020_update.p','rb'))

new_umls_2020 = [x[0] for x in new_umls_2020]
new_umls_2020 = pd.DataFrame(new_umls_2020,columns=['auis'])
new_umls_2020['lexlm_2000-NN_auis']  = nearest_neighbors_auis
new_umls_2020['lexlm_2000-NN_dist']  = list(nearest_neighbors_dist)

umls2020AB_df = umls2020AB_df.merge(new_umls_2020,on='auis',how='inner')

query_synonym_auis = list(umls2020AB_df['2020AA_synonyms'])
nearest_neighbors_auis = umls2020AB_df['lexlm_2000-NN_auis']

#Calculating Recall @ 1,5,10,50,100
recall_array = []
# closest_dist_true = []
# closest_dist_false = []

for true_syn, top100 in tqdm(zip(query_synonym_auis, nearest_neighbors_auis)):
    
    true_syn = set(true_syn)
    
    if len(true_syn) > 0:
        recalls = []

        for n in [1,5,10,50,100,200,500,1000,2000]:

            topn = set(top100[:n])
            true_pos = topn.intersection(true_syn)

            recalls.append(len(true_pos)/len(true_syn))

        recall_array.append(recalls)
#         closest_dist_true.append([top100_dist[0], np.mean(top100_dist)])
    else:
        recalls = []

        recall_array.append(recalls)
#         closest_dist_false.append([top100_dist[0], np.mean(top100_dist)])

umls2020AB_df['lexlm_2000-NN_recall'] = recall_array

In [3]:
aui_info = []

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRCONSO.RRF','r') as fp:
    
    pbar = tqdm(total=15000000)
    line = fp.readline()
    
    while line:
        line = line.split('|')
        cui = line[0]
        aui = line[7]
        string = line[-5]
        scui = line[9]
        source = line[11]
        term_status = line[2]
        string_type = line[4]
        language = line[1]
        
        aui_info.append((aui,cui,string,scui,source, term_status, string_type, language))
        
        line = fp.readline()
        pbar.update(1)

15218468it [00:55, 300868.72it/s]                                                                                                      

In [4]:
cui2sg = {}

with open('/data/Bodenreider_UMLS_DL/UMLS_VERSIONS/2020AB-ACTIVE/META/MRSTY.RRF','r') as fp:
    
    for line in fp.readlines():
        line = line.split('|')
        cui = line[0]
        sg = line[3]
        cui2sg[cui] = sg
        
original_umls = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/INTERSECT_AUI2ID.PICKLE','rb'))
new_auis = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/UNIQUE_AUI2ID.PICKLE','rb'))

# aui_vecs  = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/AUI2LAYER.PICKLE','rb'))
all_2020_auis = set(original_umls.keys()).union(new_auis.keys())

In [5]:
cui2aui = {}
aui2cui = {}
aui2scui = {}
cui2preferred = {}

aui2str = {}
str2aui = {}
aui2sg = {}
scui2auis = {}

cui_sg = []
cui_aui = []

for tup in tqdm(aui_info):
    
    tup = {'AUI':tup[0], 'CUI':tup[1], 'STR':tup[2], 'SCUI':tup[3]+'|||'+tup[4], 'SOURCE':tup[4], 'PREF':(tup[5], tup[6]), 'LANG':tup[7]}

    current_time = time.time()
    
    aui = tup['AUI']
    scui = tup['SCUI']
    
    auis = scui2auis.get(scui, [])
    auis.append(aui)
    scui2auis[scui] = auis
    
    aui2scui[aui] = scui
    
    cui = tup['CUI']
    string = tup['STR']
    
    pref = tup['PREF']
    
    if pref[0] == 'P' and pref[1] == 'PF' and tup['LANG'] == 'ENG':
        cui2preferred[cui] = string

    if aui in all_2020_auis:
        scui = tup['SCUI']
        sg = cui2sg[cui]

        auis = cui2aui.get(cui, [])
        auis.append(aui)
        cui2aui[cui] = auis
        
        aui2cui[aui] = cui
        aui2str[aui] = string
        aui2sg[aui] = sg

        auis = str2aui.get(string, [])
        auis.append(aui)
        str2aui[string] = auis
        
        cui_sg.append((cui, sg))
        cui_aui.append((cui, aui))
        
        if (time.time() - current_time) > 5:
            print(tup)
        
semgroups = pd.read_csv('SemGroups.txt',sep='|',header=None)

semtype2sg = {}

for i, row in semgroups.iterrows():
    
    st = row[3]
    sg = row[1]
    
    semtype2sg[st] = sg
    
cuis = []
sts = []

for aui in umls2020AB_df.auis:
    
    cuis.append(aui2cui[aui])
    sts.append(aui2sg[aui])
    
umls2020AB_df['cuis'] = cuis
umls2020AB_df['sem_types'] = sts
umls2020AB_df['sem_groups'] = [semtype2sg[st] for st in sts]


  0%|                                                                                                     | 0/15229292 [00:00<?, ?it/s]
15229292it [01:10, 300868.72it/s]                                                            | 4454/15229292 [00:01<59:08, 4289.90it/s]
  8%|██████▌                                                                            | 1212798/15229292 [00:22<02:05, 111710.63it/s]


 16%|████████████▉                                                                      | 2379648/15229292 [00:34<01:02, 205767.49it/s]


 23%|███████████████████▎                                                               | 3541694/15229292 [00:46<00:58, 200582.54it/s]


 31%|█████████████████████████▍                                                         | 4674405/15229292 [00:59<01:11, 147721.68it/s]


 38%|███████████████████████████████▌                                                   | 5792833/15229292 [01:09<00:53, 177283.04it/s]


...

{'AUI': 'A27427942', 'CUI': 'C4294922', 'STR': 'retracted pars tensa bilateral with drainage bloody', 'SCUI': '299265|||MEDCIN', 'SOURCE': 'MEDCIN', 'PREF': ('P', 'PF'), 'LANG': 'ENG'}



 94%|█████████████████████████████████████████████████████████████████████████████▏    | 14327927/15229292 [02:44<00:05, 157623.84it/s]


 99%|█████████████████████████████████████████████████████████████████████████████████▌| 15140424/15229292 [02:51<00:00, 125580.87it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [02:51<00:00, 88638.89it/s]


NameError: name 'umls2020AB_df' is not defined

In [ ]:
str2cuis = {}

for aui in tqdm(aui2str.keys()):
    
    aui_string = aui2str[aui]
    
    cuis = str2cuis.get(aui_string, set())
    cuis.add(aui2cui[aui])
    str2cuis[aui_string] = cuis

In [ ]:
str2cui_df = pd.DataFrame(str2cuis.items())

In [ ]:
str2cui_df['num_cuis'] = [len(c) for c in str2cui_df[1]]

In [ ]:
str2cui_df.sort_values('num_cuis',ascending=False)[:100]

In [18]:
sem_groups = []

for i,row in tqdm(str2cui_df.iterrows(), total=len(str2cui_df)):
    cuis = row[1]
    
    group_row = set()
    
    for cui in cuis:
        group = cui2sg[cui]
        group_row.add(group)
        
    sem_groups.append(group_row)


0it [00:00, ?it/s]
1it [00:01,  1.12s/it]
1397it [00:01, 1574.38it/s]
2775it [00:01, 3267.13it/s]
4157it [00:01, 5012.07it/s]
5536it [00:01, 6689.11it/s]
6798it [00:01, 7443.84it/s]
7955it [00:01, 7977.90it/s]
9307it [00:01, 9268.00it/s]
10698it [00:01, 10426.02it/s]
12097it [00:02, 11362.00it/s]
13495it [00:02, 12073.35it/s]
14870it [00:02, 12540.91it/s]
16259it [00:02, 12925.71it/s]
17646it [00:02, 13197.63it/s]
19028it [00:02, 13378.26it/s]
20430it [00:02, 13565.42it/s]
21815it [00:02, 13649.12it/s]
23202it [00:02, 13712.03it/s]
24589it [00:02, 13756.54it/s]
25973it [00:03, 13755.27it/s]
27366it [00:03, 13806.65it/s]
28762it [00:03, 13850.64it/s]
30150it [00:03, 13747.82it/s]
31532it [00:03, 13767.16it/s]
32920it [00:03, 13800.34it/s]
34306it [00:03, 13817.55it/s]
35689it [00:03, 13795.39it/s]
37073it [00:03, 13807.39it/s]
38458it [00:03, 13818.54it/s]
39853it [00:04, 13854.95it/s]
41239it [00:04, 13780.04it/s]
42623it [00:04, 13797.35it/s]
44014it [00:04, 13828.85it/s]
45398it [00

667119it [00:49, 13702.90it/s]
668490it [00:49, 13701.12it/s]
669864it [00:49, 13711.45it/s]
671242it [00:50, 13731.27it/s]
672616it [00:50, 13607.87it/s]
673994it [00:50, 13657.08it/s]
675375it [00:50, 13702.48it/s]
676746it [00:50, 13696.75it/s]
678116it [00:50, 13634.64it/s]
679482it [00:50, 13639.99it/s]
680847it [00:50, 13604.84it/s]
682219it [00:50, 13638.17it/s]
683583it [00:50, 13601.08it/s]
684954it [00:51, 13631.41it/s]
686334it [00:51, 13680.96it/s]
687718it [00:51, 13726.87it/s]
689091it [00:51, 13622.42it/s]
690468it [00:51, 13665.63it/s]
691841it [00:51, 13683.33it/s]
693212it [00:51, 13690.00it/s]
694582it [00:51, 13685.85it/s]
695961it [00:51, 13715.21it/s]
697338it [00:51, 13729.23it/s]
698717it [00:52, 13746.74it/s]
700092it [00:52, 13724.04it/s]
701472it [00:52, 13745.83it/s]
702847it [00:52, 13734.16it/s]
704221it [00:52, 13724.14it/s]
705594it [00:52, 13678.05it/s]
706970it [00:52, 13700.44it/s]
708341it [00:52, 13683.39it/s]
709710it [00:52, 13658.19it/s]
711092it

1317318it [01:37, 13650.94it/s]
1318684it [01:37, 13646.57it/s]
1320064it [01:37, 13689.73it/s]
1321434it [01:37, 13678.25it/s]
1322803it [01:38, 13679.83it/s]
1324173it [01:38, 13685.70it/s]
1325553it [01:38, 13719.29it/s]
1326925it [01:38, 13602.83it/s]
1328294it [01:38, 13627.28it/s]
1329674it [01:38, 13678.30it/s]
1331042it [01:38, 13655.99it/s]
1332412it [01:38, 13666.92it/s]
1333785it [01:38, 13682.91it/s]
1335154it [01:38, 13663.19it/s]
1336527it [01:39, 13666.08it/s]
1337907it [01:39, 13705.52it/s]
1339283it [01:39, 13718.90it/s]
1340655it [01:39, 13688.97it/s]
1342024it [01:39, 13651.72it/s]
1343390it [01:39, 13627.94it/s]
1344761it [01:39, 13650.72it/s]
1346141it [01:39, 13694.76it/s]
1347511it [01:39, 13634.98it/s]
1348883it [01:39, 13658.66it/s]
1350257it [01:40, 13682.53it/s]
1351626it [01:40, 13644.55it/s]
1352991it [01:40, 13629.53it/s]
1354360it [01:40, 13646.92it/s]
1355727it [01:40, 13653.16it/s]
1357095it [01:40, 13659.43it/s]
1358461it [01:40, 13599.71it/s]
1359829i

1970968it [02:24, 14172.48it/s]
1972388it [02:24, 14178.24it/s]
1973806it [02:25, 14159.83it/s]
1975224it [02:25, 14164.75it/s]
1976661it [02:25, 14224.93it/s]
1978084it [02:25, 14225.41it/s]
1979513it [02:25, 14244.33it/s]
1980938it [02:25, 14218.12it/s]
1982360it [02:25, 14206.52it/s]
1983786it [02:25, 14220.92it/s]
1985209it [02:25, 14216.14it/s]
1986631it [02:25, 14203.77it/s]
1988060it [02:26, 14229.44it/s]
1989483it [02:26, 14224.08it/s]
1990906it [02:26, 14192.70it/s]
1992326it [02:26, 14184.35it/s]
1993757it [02:26, 14221.12it/s]
1995186it [02:26, 14239.99it/s]
1996611it [02:26, 14212.50it/s]
1998039it [02:26, 14232.30it/s]
1999463it [02:26, 14217.50it/s]
2000887it [02:26, 14222.08it/s]
2002310it [02:27, 14192.26it/s]
2003730it [02:27, 14179.67it/s]
2005154it [02:27, 14195.95it/s]
2006574it [02:27, 14192.88it/s]
2007994it [02:27, 14145.61it/s]
2009420it [02:27, 14178.05it/s]
2010846it [02:27, 14200.07it/s]
2012283it [02:27, 14248.41it/s]
2013708it [02:27, 13831.12it/s]
2015112i

2637435it [03:12, 14237.96it/s]
2638864it [03:12, 14252.63it/s]
2640295it [03:12, 14266.84it/s]
2641726it [03:12, 14279.50it/s]
2643154it [03:12, 14264.66it/s]
2644581it [03:12, 14265.69it/s]
2646008it [03:12, 14253.91it/s]
2647434it [03:12, 14252.55it/s]
2648860it [03:12, 14197.58it/s]
2650292it [03:12, 14233.27it/s]
2651728it [03:13, 14267.83it/s]
2653155it [03:13, 14221.82it/s]
2654595it [03:13, 14273.60it/s]
2656023it [03:13, 14273.89it/s]
2657456it [03:13, 14288.54it/s]
2658885it [03:13, 14265.61it/s]
2660319it [03:13, 14287.38it/s]
2661748it [03:13, 14283.48it/s]
2663178it [03:13, 14286.42it/s]
2664607it [03:13, 14217.24it/s]
2666051it [03:14, 14283.32it/s]
2667481it [03:14, 14287.82it/s]
2668921it [03:14, 14319.24it/s]
2670353it [03:14, 14184.84it/s]
2671784it [03:14, 14220.07it/s]
2673231it [03:14, 14293.55it/s]
2674661it [03:14, 14282.95it/s]
2676090it [03:14, 14259.68it/s]
2677517it [03:14, 14255.00it/s]
2678943it [03:14, 14255.85it/s]
2680377it [03:15, 14278.18it/s]
2681805i

...

4643020it [06:56, 14424.01it/s]
4644463it [06:56, 14405.66it/s]
4645920it [06:56, 14452.97it/s]
4647373it [06:57, 14475.27it/s]
4648828it [06:57, 14496.22it/s]
4650287it [06:57, 14521.92it/s]
4651743it [06:57, 14530.93it/s]
4653197it [06:57, 14519.05it/s]
4654649it [06:57, 14357.94it/s]
4656092it [06:57, 14378.18it/s]
4657551it [06:57, 14440.64it/s]
4659001it [06:57, 14456.76it/s]
4660447it [06:57, 14456.68it/s]
4661893it [06:58, 14344.01it/s]
4663348it [06:58, 14403.14it/s]
4664811it [06:58, 14470.41it/s]
4666261it [06:58, 14477.33it/s]
4667709it [06:58, 14357.02it/s]
4669159it [06:58, 14397.23it/s]
4670619it [06:58, 14456.24it/s]
4672072it [06:58, 14477.82it/s]
4673526it [06:58, 14495.50it/s]
4674976it [06:59, 14368.18it/s]
4676431it [06:59, 14420.90it/s]
4677880it [06:59, 14440.28it/s]
4679325it [06:59, 14376.86it/s]
4680784it [06:59, 14438.38it/s]
4682228it [06:59, 14437.01it/s]
4683681it [06:59, 14463.72it/s]
4685128it [06:59, 14400.74it/s]
4686587it [06:59, 14455.91it/s]
4688044i

5318634it [07:43, 14324.08it/s]
5320077it [07:43, 14353.45it/s]
5321522it [07:44, 14379.49it/s]
5322961it [07:44, 14306.24it/s]
5324396it [07:44, 14318.10it/s]
5325851it [07:44, 14385.20it/s]
5327302it [07:44, 14420.58it/s]
5328745it [07:44, 14392.37it/s]
5330196it [07:44, 14425.64it/s]
5331647it [07:44, 14450.64it/s]
5333093it [07:44, 14377.37it/s]
5334532it [07:44, 14379.95it/s]
5335982it [07:45, 14413.73it/s]
5337426it [07:45, 14421.27it/s]
5338869it [07:45, 14396.28it/s]
5340309it [07:45, 14384.01it/s]
5341750it [07:45, 14391.12it/s]
5343190it [07:45, 14373.73it/s]
5344628it [07:45, 14366.82it/s]
5346065it [07:45, 14365.68it/s]
5347504it [07:45, 14370.08it/s]
5348942it [07:45, 14332.99it/s]
5350376it [07:46, 14306.19it/s]
5351829it [07:46, 14372.08it/s]
5353279it [07:46, 14409.48it/s]
5354720it [07:46, 14405.13it/s]
5356161it [07:46, 14396.62it/s]
5357601it [07:46, 14376.81it/s]
5359039it [07:46, 14370.65it/s]
5360479it [07:46, 14379.11it/s]
5361926it [07:46, 14404.99it/s]
5363367i

5968252it [08:31, 13856.88it/s]
5969638it [08:31, 13853.50it/s]
5971024it [08:31, 13801.46it/s]
5972406it [08:31, 13804.69it/s]
5973790it [08:31, 13814.02it/s]
5975173it [08:31, 13817.86it/s]
5976555it [08:31, 13782.10it/s]
5977942it [08:31, 13806.72it/s]
5979323it [08:31, 13768.39it/s]
5980700it [08:31, 13735.06it/s]
5982097it [08:32, 13802.25it/s]
5983486it [08:32, 13827.48it/s]
5984877it [08:32, 13850.95it/s]
5986263it [08:32, 13747.02it/s]
5987638it [08:32, 13740.27it/s]
5989019it [08:32, 13760.61it/s]
5990403it [08:32, 13782.35it/s]
5991782it [08:32, 13767.12it/s]
5993160it [08:32, 13770.41it/s]
5994538it [08:32, 13768.60it/s]
5995927it [08:33, 13802.52it/s]
5997308it [08:33, 13785.98it/s]
5998688it [08:33, 13789.43it/s]
6000067it [08:33, 13767.34it/s]
6001448it [08:33, 13779.90it/s]
6002827it [08:33, 13720.40it/s]
6004202it [08:33, 13727.72it/s]
6005591it [08:33, 13775.96it/s]
6006972it [08:33, 13785.51it/s]
6008351it [08:33, 13715.28it/s]
6009742it [08:34, 13771.47it/s]
6011132i

6636106it [09:18, 14353.77it/s]
6637542it [09:18, 14291.42it/s]
6638980it [09:18, 14316.62it/s]
6640412it [09:18, 14312.74it/s]
6641845it [09:18, 14315.42it/s]
6643277it [09:18, 14273.44it/s]
6644705it [09:18, 14273.35it/s]
6646148it [09:18, 14319.97it/s]
6647581it [09:18, 14289.43it/s]
6649010it [09:19, 14123.73it/s]
6650436it [09:19, 14162.35it/s]
6651884it [09:19, 14255.12it/s]
6653322it [09:19, 14291.84it/s]
6654756it [09:19, 14303.98it/s]
6656187it [09:19, 14272.51it/s]
6657621it [09:19, 14290.85it/s]
6659057it [09:19, 14310.29it/s]
6660492it [09:19, 14321.31it/s]
6661931it [09:20, 14341.55it/s]
6663366it [09:20, 14333.30it/s]
6664800it [09:20, 14332.17it/s]
6666234it [09:20, 14176.42it/s]
6667656it [09:20, 14186.63it/s]
6669090it [09:20, 14229.33it/s]
6670524it [09:20, 14261.23it/s]
6671956it [09:20, 14277.60it/s]
6673384it [09:20, 14248.00it/s]
6674811it [09:20, 14251.63it/s]
6676237it [09:21, 14236.91it/s]
6677670it [09:21, 14263.74it/s]
6679097it [09:21, 13874.91it/s]
6680537i

7298277it [10:05, 13655.96it/s]
7299660it [10:05, 13707.56it/s]
7301047it [10:05, 13755.61it/s]
7302423it [10:05, 13703.43it/s]
7303806it [10:05, 13739.25it/s]
7305203it [10:05, 13805.56it/s]
7306584it [10:06, 13802.83it/s]
7307965it [10:06, 13659.08it/s]
7309354it [10:06, 13725.73it/s]
7310741it [10:06, 13766.61it/s]
7312125it [10:06, 13788.20it/s]
7313510it [10:06, 13805.42it/s]
7314891it [10:06, 13673.54it/s]
7316277it [10:06, 13727.16it/s]
7317662it [10:06, 13762.66it/s]
7319057it [10:06, 13815.65it/s]
7320446it [10:07, 13836.44it/s]
7321838it [10:07, 13839.65it/s]
7323226it [10:07, 13851.45it/s]
7324612it [10:07, 13843.62it/s]
7325998it [10:07, 13846.54it/s]
7327390it [10:07, 13865.83it/s]
7328777it [10:07, 13851.62it/s]
7330163it [10:07, 13835.44it/s]
7331547it [10:07, 13540.51it/s]
7332919it [10:07, 13592.38it/s]
7334306it [10:08, 13673.88it/s]
7335691it [10:08, 13725.15it/s]
7337079it [10:08, 13768.65it/s]
7338457it [10:08, 13628.45it/s]
7339838it [10:08, 13681.99it/s]
7341226i

In [19]:
str2cui_df['sem_groups'] = ['|||'.join([str(s) for s in np.sort(list(gs),kind='stable')]) for gs in tqdm(sem_groups)]


 23%|███████████████████▏                                                                | 1808194/7931549 [00:11<00:45, 134831.46it/s]


 45%|██████████████████████████████████████                                              | 3593178/7931549 [00:23<00:27, 155615.15it/s]


 68%|████████████████████████████████████████████████████████▉                           | 5380052/7931549 [00:35<00:16, 151907.17it/s]


 90%|███████████████████████████████████████████████████████████████████████████▊        | 7155287/7931549 [00:47<00:05, 152324.54it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 7931549/7931549 [00:52<00:00, 151502.75it/s]


In [20]:
ambiguous_strings = str2cui_df[str2cui_df.num_cuis > 1]

In [60]:
ambiguous_strings.groupby('sem_groups').count().sort_values(0,ascending=False)[:50]

,0,1,num_cuis
sem_groups,,,
Gene or Genome,10452,10452,10452
Biologically Active Substance|||Gene or Genome,3885,3885,3885
Enzyme|||Gene or Genome,2312,2312,2312
Disease or Syndrome|||Gene or Genome,2295,2295,2295
Clinical Attribute|||Intellectual Product,1818,1818,1818
Neoplastic Process,1603,1603,1603
Clinical Drug,1343,1343,1343
Disease or Syndrome,1325,1325,1325
Clinical Attribute,1253,1253,1253


In [22]:
str2cui_df.groupby('num_cuis').count().sort_values(0,ascending=False)

,0,1,sem_groups
num_cuis,,,
1,7865805,7865805,7865805
2,54123,54123,54123
3,8063,8063,8063
4,1951,1951,1951
5,716,716,716
6,350,350,350
7,187,187,187
8,120,120,120
9,63,63,63


In [23]:
source_syn_candidates = []
source_syn_plus = []

for aui in tqdm(umls2020AB_df.auis):
    scui = aui2scui[aui]
    
    if scui.split('|||')[0] != '':       
        #Only get terms with source synonymy
        source_syns = list(set(scui2auis[scui]))
        if len(source_syns) > 500:
            break
            
        #For each source synonym, get all its 2020AA defined synonyms and add them to the candidate list
        all_syns = copy.deepcopy(source_syns)
        for source_syn_aui in source_syns:
            if source_syn_aui in aui2cui:
                AA_syns = cui2aui[aui2cui[source_syn_aui]]
                all_syns.extend(AA_syns)
    else:
        source_syns = []
        all_syns = []
        
    source_syn_candidates.append(source_syns)
    source_syn_plus.append(all_syns)
    
umls2020AB_df['source_syns'] = source_syn_candidates
umls2020AB_df['source_syns_plus'] = source_syn_plus


100%|███████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [00:05<00:00, 80247.64it/s]


## Evaluating kNN Models + Source Synonymy 

In [24]:
aui_columns = umls2020AB_df.filter(regex='.*_auis').columns

In [25]:
aui_columns

Index(['sapbert_2000-NN_auis'], dtype='object')

In [26]:
query_synonym_auis = list(umls2020AB_df['2020AA_synonyms'])
source_syns = umls2020AB_df['source_syns']

for aui_col in aui_columns:
    print(aui_col)
    aui_name = aui_col.split('_auis')[0]
    nearest_neighbors_auis = umls2020AB_df[aui_col]

    #Calculating Recall @ 1,5,10,50,100
    recall_array = []

    for true_syn, top100, source in tqdm(zip(query_synonym_auis, nearest_neighbors_auis, source_syns)):

        true_syn = set(true_syn)

        source = copy.deepcopy(list(set(source)))

        if source is not None:
            source_syn_num = len(source)        
            source.extend(top100)
        else:
            source = top100        
            source_syn_num = 0

        if len(true_syn) > 0:
            recalls = []

            for n in [0,1,5,10,50,100,200,500,1000,2000]:

                topn = set(source[:n+source_syn_num])
                true_pos = topn.intersection(true_syn)

                recalls.append(len(true_pos)/len(true_syn))

            recall_array.append(recalls)
        else:
            recalls = []

            recall_array.append(recalls)

    umls2020AB_df['{}_source_syn_recall'.format(aui_name)] = recall_array

sapbert_2000-NN_auis



0it [00:00, ?it/s]
585it [00:00, 5837.06it/s]
1169it [00:00, 5819.33it/s]
1841it [00:00, 6223.27it/s]
2545it [00:00, 6543.60it/s]
3200it [00:00, 6414.60it/s]
3842it [00:00, 6224.71it/s]
4522it [00:00, 6402.64it/s]
5164it [00:00, 6379.48it/s]
5803it [00:00, 6294.74it/s]
6434it [00:01, 6099.33it/s]
7130it [00:01, 6348.30it/s]
7767it [00:01, 6352.69it/s]
8404it [00:01, 6282.96it/s]
9034it [00:01, 6158.94it/s]
9677it [00:01, 6235.28it/s]
10302it [00:01, 6238.58it/s]
10927it [00:01, 6003.49it/s]
11856it [00:01, 6952.81it/s]
12557it [00:02, 5724.15it/s]
13293it [00:02, 6139.66it/s]
13942it [00:02, 6099.06it/s]
14576it [00:02, 5520.99it/s]
15164it [00:02, 5611.41it/s]
15744it [00:02, 5655.84it/s]
16324it [00:02, 5238.20it/s]
16882it [00:02, 5329.24it/s]
17426it [00:02, 5280.63it/s]
17962it [00:03, 5026.85it/s]
18593it [00:03, 5376.83it/s]
19139it [00:03, 5329.72it/s]
19678it [00:03, 5190.81it/s]
20202it [00:03, 5050.70it/s]
20878it [00:03, 5529.64it/s]
21437it [00:03, 5379.09it/s]
21979it [0

392255it [01:01, 5383.45it/s]
392874it [01:01, 4797.01it/s]
393417it [01:01, 4126.60it/s]
393884it [01:02, 3702.66it/s]
394293it [01:02, 3454.76it/s]
394664it [01:02, 3315.79it/s]
395011it [01:02, 3261.69it/s]
395347it [01:02, 3140.40it/s]
395667it [01:02, 3069.33it/s]
395977it [01:02, 2972.95it/s]
396276it [01:02, 2956.14it/s]
396591it [01:03, 3006.44it/s]
396905it [01:03, 3041.79it/s]
397211it [01:03, 3037.45it/s]
397529it [01:03, 3076.69it/s]
397838it [01:03, 3071.94it/s]
398158it [01:03, 3107.51it/s]
398481it [01:03, 3142.21it/s]
398796it [01:03, 3105.60it/s]
399353it [01:03, 3829.52it/s]
399738it [01:03, 3586.34it/s]
400126it [01:04, 3668.76it/s]
400497it [01:04, 3464.55it/s]
400848it [01:04, 3284.74it/s]
401181it [01:04, 3179.67it/s]
401502it [01:04, 3144.20it/s]
401819it [01:04, 3036.67it/s]
402125it [01:04, 2907.86it/s]
402427it [01:04, 2937.47it/s]
402730it [01:04, 2961.34it/s]
403028it [01:05, 2951.26it/s]
403324it [01:05, 2588.01it/s]
403593it [01:05, 2614.96it/s]
403908it [

In [27]:
query_synonym_auis = list(umls2020AB_df['2020AA_synonyms'])
source_syns = umls2020AB_df['source_syns_plus']

for aui_col in aui_columns:
    print(aui_col)

    aui_name = aui_col.split('_auis')[0]
    nearest_neighbors_auis = umls2020AB_df[aui_col]

    #Calculating Recall @ 1,5,10,50,100
    recall_array = []

    for true_syn, top100, source in tqdm(zip(query_synonym_auis, nearest_neighbors_auis, source_syns),total=len(query_synonym_auis)):

        true_syn = set(true_syn)

        source = copy.deepcopy(list(set(source)))

        if source is not None:
            source_syn_num = len(source)        
            source.extend(top100)
        else:
            source = top100        
            source_syn_num = 0

        if len(true_syn) > 0:
            recalls = []

            for n in [0,1,5,10,50,100,200,500,1000,2000]:

                topn = set(source[:n+source_syn_num])
                true_pos = topn.intersection(true_syn)

                recalls.append(len(true_pos)/len(true_syn))

            recall_array.append(recalls)
        else:
            recalls = []

            recall_array.append(recalls)

    umls2020AB_df['{}_source_syn_plus_recall'.format(aui_name)] = recall_array

sapbert_2000-NN_auis



 19%|████████████████▋                                                                        | 80471/430135 [00:13<01:03, 5495.07it/s]


 42%|█████████████████████████████████████▍                                                  | 182700/430135 [00:28<01:00, 4095.78it/s]


 66%|██████████████████████████████████████████████████████████▏                             | 284617/430135 [00:43<00:23, 6091.09it/s]


 85%|██████████████████████████████████████████████████████████████████████████▉             | 366412/430135 [00:57<00:12, 5056.91it/s]


 99%|███████████████████████████████████████████████████████████████████████████████████████ | 425333/430135 [01:11<00:01, 2755.44it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [01:13<00:00, 5856.23it/s]


In [28]:
recall_df = []
names = []

for recall_col in umls2020AB_df.filter(regex='.*recall.*').columns:
    names.append(recall_col)
    recall_array = list(umls2020AB_df[recall_col].values)
    recall_df.append(pd.DataFrame(recall_array).agg(['mean']))
    

recall_df = pd.concat(recall_df)
recall_df['model'] = names

recall_df

,0,1,2,3,4,5,6,7,8,9,model
mean,0.20,0.44,0.53,0.71,0.76,0.81,0.86,0.88,0.89,NaN,sapbert_2000-NN_recall
mean,0.35,0.46,0.63,0.70,0.83,0.86,0.90,0.93,0.94,0.95,sapbert_2000-NN_source_syn_recall
mean,0.83,0.85,0.89,0.90,0.94,0.95,0.97,0.99,0.99,0.99,sapbert_2000-NN_source_syn_plus_recall


## RW-UVA Using CUIs

In [29]:
query_synonym_auis = list(umls2020AB_df['2020AA_synonyms'])
umls2020AB_df['2020AA_synonyms_cuis'] = [[aui2cui[aui] for aui in true_syn] for true_syn in tqdm(query_synonym_auis)]


100%|██████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [00:01<00:00, 332160.53it/s]


In [30]:
source_syns = umls2020AB_df['source_syns']

source_syn_cuis = []

for source_syn_row in tqdm(source_syns):
    
    source_syn_row_2020AA = []
    
    for source_syn_aui in source_syn_row:
        if source_syn_aui in aui2cui:
            source_syn_row_2020AA.append(aui2cui[source_syn_aui])
    
    source_syn_cuis.append(source_syn_row_2020AA)
    
umls2020AB_df['source_syns_cuis'] = source_syn_cuis


100%|██████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [00:01<00:00, 319182.24it/s]


In [31]:
for aui_col in aui_columns:
    aui_name = aui_col.split('_auis')[0]
    nearest_neighbors_auis = umls2020AB_df[aui_col]
    umls2020AB_df['{}_cuis'.format(aui_name)] = [[aui2cui[aui] for aui in pred_syn] for pred_syn in tqdm(nearest_neighbors_auis)]


  3%|██▋                                                                                       | 12845/430135 [00:13<07:42, 902.56it/s]


  6%|█████▏                                                                                    | 25083/430135 [00:26<06:51, 983.69it/s]


  9%|████████                                                                                  | 38281/430135 [00:41<07:26, 876.68it/s]


 12%|██████████▍                                                                               | 49727/430135 [00:54<06:57, 910.29it/s]


 14%|████████████▉                                                                             | 62109/430135 [01:09<08:26, 726.74it/s]


...

 92%|█████████████████████████████████████████████████████████████████████████████████▋       | 394951/430135 [08:48<00:51, 677.06it/s]


 94%|███████████████████████████████████████████████████████████████████████████████████▉     | 405903/430135 [09:01<00:24, 977.26it/s]


 97%|██████████████████████████████████████████████████████████████████████████████████████   | 416073/430135 [09:15<00:19, 718.40it/s]


 99%|████████████████████████████████████████████████████████████████████████████████████████▏| 425951/430135 [09:29<00:05, 804.64it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [09:35<00:00, 747.64it/s]


In [32]:
cui_columns = umls2020AB_df.filter(regex='.*2000.*_cuis').columns

In [33]:
cui_columns

Index(['sapbert_2000-NN_cuis'], dtype='object')

In [34]:
query_synonym_cuis = list(umls2020AB_df['2020AA_synonyms_cuis'])

for cui_col in cui_columns:
    print(cui_col)
    cui_name = cui_col.split('_cuis')[0]
    nearest_neighbors_cuis = umls2020AB_df[cui_col]

    #Calculating Recall @ 1,5,10,50,100
    recall_array = []

    for true_syn, top100 in tqdm(zip(query_synonym_cuis, nearest_neighbors_cuis), total=len(query_synonym_cuis)):

        true_syn = set(true_syn)

        if len(true_syn) > 0:
            recalls = []

            for n in [1,5,10,50,100,200,500,1000,2000]:

                topn = set(top100[:n])
                true_pos = topn.intersection(true_syn)

                recalls.append(len(true_pos)/len(true_syn))

            recall_array.append(recalls)
        else:
            recalls = []

            recall_array.append(recalls)

    umls2020AB_df['{}_cui_recall'.format(cui_name)] = recall_array

sapbert_2000-NN_cuis



 29%|█████████████████████████▋                                                              | 125723/430135 [00:17<00:50, 5977.88it/s]


 71%|██████████████████████████████████████████████████████████████▍                         | 305346/430135 [00:37<00:23, 5335.01it/s]


 93%|█████████████████████████████████████████████████████████████████████████████████▊      | 399763/430135 [00:52<00:08, 3518.77it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [01:00<00:00, 7076.10it/s]


In [35]:
query_synonym_cuis = list(umls2020AB_df['2020AA_synonyms_cuis'])
source_syns = umls2020AB_df['source_syns_cuis']

for cui_col in cui_columns:
    print(cui_col)
    cui_name = cui_col.split('_cuis')[0]
    nearest_neighbors_cuis = umls2020AB_df[cui_col]
    
    #Calculating Recall @ 1,5,10,50,100
    recall_array = []

    for true_syn, top100, source in tqdm(zip(query_synonym_cuis, nearest_neighbors_cuis, source_syns), total=len(query_synonym_cuis)):

        true_syn = set(true_syn)
        source = copy.deepcopy(list(set(source)))

        if len(true_syn) > 0:
            recalls = []

            if source is not None:
                source_syn_num = len(source)        
                source.extend(top100)
            else:
                source = top100
                source_syn_num = 0

            for n in [0,1,5,10,50,100,200,500,1000,2000]:
                topn = set(source[:n+source_syn_num])
                true_pos = topn.intersection(true_syn)

                recalls.append(len(true_pos)/len(true_syn))

            recall_array.append(recalls)
        else:
            recalls = []

            recall_array.append(recalls)

    umls2020AB_df['{}_source_syn_cui_recall'.format(cui_name)] = recall_array

sapbert_2000-NN_cuis



 27%|███████████████████████▊                                                               | 117609/430135 [00:16<00:27, 11462.85it/s]


 65%|████████████████████████████████████████████████████████▎                              | 278640/430135 [00:37<00:14, 10479.59it/s]


 87%|████████████████████████████████████████████████████████████████████████████▎           | 373300/430135 [00:52<00:11, 5158.99it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [01:07<00:00, 6365.97it/s]


In [36]:
umls2020AB_df['number_source_syn_cuis'] = [len(set(c)) for c in umls2020AB_df.source_syns_cuis] 
umls2020AB_df['number_source_syn_auis'] = [len(set(c)) for c in umls2020AB_df.source_syns] 
umls2020AB_df['number_source_syn_plus_auis'] = [len(set(c)) for c in umls2020AB_df.source_syns_plus] 

In [37]:
umls2020AB_df.describe()

,0,num_syms,number_source_syn_cuis,number_source_syn_auis,number_source_syn_plus_auis
count,430135.00,430135.00,430135.00,430135.00,430135.00
mean,11.46,7.50,0.89,2.90,5.23
std,9.86,35.94,0.34,3.84,10.92
min,3.00,0.00,0.00,0.00,0.00
25%,7.00,0.00,1.00,1.00,1.00
50%,10.00,0.00,1.00,1.00,2.00
75%,13.00,4.00,1.00,4.00,6.00
max,1104.00,550.00,7.00,65.00,2714.00


In [38]:
recall_df = []
names = []

for recall_col in umls2020AB_df.filter(regex='.*recall.*').columns:
    names.append(recall_col)
    recall_array = list(umls2020AB_df[recall_col].values)
    recall_df.append(pd.DataFrame(recall_array).agg(['mean']))

recall_df = pd.concat(recall_df)
recall_df['model'] = names

recall_df

,0,1,2,3,4,5,6,7,8,9,model
mean,0.20,0.44,0.53,0.71,0.76,0.81,0.86,0.88,0.89,NaN,sapbert_2000-NN_recall
mean,0.35,0.46,0.63,0.70,0.83,0.86,0.90,0.93,0.94,0.95,sapbert_2000-NN_source_syn_recall
mean,0.83,0.85,0.89,0.90,0.94,0.95,0.97,0.99,0.99,0.99,sapbert_2000-NN_source_syn_plus_recall
mean,0.72,0.83,0.86,0.91,0.92,0.93,0.94,0.95,0.95,NaN,sapbert_2000-NN_cui_recall
mean,0.83,0.96,0.98,0.99,0.99,0.99,1.00,1.00,1.00,1.00,sapbert_2000-NN_source_syn_cui_recall


In [39]:
cui_columns

Index(['sapbert_2000-NN_cuis'], dtype='object')

In [40]:
umls2020AB_df['2020AA_synonyms_cuis']

8768549                                                   []
8769337                                                   []
8949912                                                   []
8763774                                                   []
8764953                                                   []
                                 ...                        
8529453    [C0042679, C0042679, C0042679, C0042679, C0042...
8529270    [C0041984, C0041984, C0041984, C0041984, C0041...
8529351    [C0042232, C0042232, C0042232, C0042232, C0042...
8601787    [C1425082, C1425082, C1425082, C1425082, C1425...
8940025                                                   []
Name: 2020AA_synonyms_cuis, Length: 430135, dtype: object

In [41]:
umls2020AB_df.source_syns_cuis

8768549                      []
8769337                      []
8949912                      []
8763774                      []
8764953                      []
                   ...         
8529453    [C0042679, C0042679]
8529270              [C0041984]
8529351    [C0042232, C0042232]
8601787                      []
8940025                      []
Name: source_syns_cuis, Length: 430135, dtype: object

In [42]:
umls2020AB_df['sapbert_2000-NN_cuis']

8768549    [C4058663, C5140346, C3818349, C3818349, C3818...
8769337    [C5207314, C5135790, C5136582, C4689308, C5136...
8949912    [C4046595, C4530008, C4291206, C4551034, C5228...
8763774    [C4519986, C3555441, C4056238, C4689534, C4056...
8764953    [C4735039, C4291270, C4519638, C3665116, C4522...
                                 ...                        
8529453    [C0042679, C0042679, C0042679, C0042679, C0042...
8529270    [C0041984, C0041984, C0041984, C0041984, C0041...
8529351    [C0042232, C1519910, C0042232, C0042232, C0042...
8601787    [C2827459, C3810343, C3538738, C4014722, C1425...
8940025    [C1555605, C2677109, C0018553, C1705310, C4684...
Name: sapbert_2000-NN_cuis, Length: 430135, dtype: object

In [43]:
#Getting Errors and Fine Tuning Dataset
all_candidates = []
query_synonym_cuis = list(umls2020AB_df['2020AA_synonyms_cuis'])
source_syns = umls2020AB_df['source_syns_cuis']

for cui_col in cui_columns:
    print(cui_col)
    cui_name = cui_col.split('_cuis')[0]
    nearest_neighbors_cuis = umls2020AB_df[cui_col]
    
    pair_dataset = []
    pair_labels = []
    errors = []

    for true_syn, topn, source in tqdm(zip(query_synonym_cuis, nearest_neighbors_cuis, source_syns), total=len(query_synonym_cuis)):

        if source is not None:
            source_syn_num = len(source)        
        else:
            source_syn_num = 0

        true_syn = set(true_syn)
        source_plus_top = copy.deepcopy(list(set(source)))
        source_plus_top.extend(topn)        
        
        all_candidates.append((true_syn, source_plus_top, [c in true_syn for c in source_plus_top]))
        
        missing_syns = []
        for n in [0,1,5,10,50,100,200,500,1000,2000]:
            topn = set(source_plus_top[:n+source_syn_num])
            errorsn = true_syn.difference(topn)
            
            missing_syns.append(errorsn)
            
        errors.append(missing_syns)

sapbert_2000-NN_cuis



  4%|███▋                                                                                     | 18021/430135 [00:12<04:58, 1379.73it/s]


  9%|███████▋                                                                                 | 37113/430135 [00:26<04:07, 1587.25it/s]


 13%|███████████▎                                                                            | 55431/430135 [02:50<18:42:54,  5.56it/s]


 17%|███████████████▎                                                                         | 74164/430135 [03:03<04:16, 1389.19it/s]


...

 85%|███████████████████████████████████████████████████████████████████████████             | 367066/430135 [07:02<00:49, 1276.86it/s]


 89%|██████████████████████████████████████████████████████████████████████████████▍         | 383551/430135 [07:15<00:37, 1256.71it/s]


 93%|█████████████████████████████████████████████████████████████████████████████████▊      | 399862/430135 [07:28<00:24, 1253.11it/s]


 97%|█████████████████████████████████████████████████████████████████████████████████████▏  | 416692/430135 [07:42<00:10, 1284.72it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 430135/430135 [07:52<00:00, 909.89it/s]


In [44]:
pickle.dump(errors, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_source_cui_errors.p','wb'))

KeyboardInterrupt: 

In [37]:
pickle.dump(all_candidates, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_candidates_cui_dataset.p','wb'))

In [41]:
pickle.dump(umls2020AB_df.cuis.values, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_cuis.p','wb'))

In [45]:
all_candidates

KeyboardInterrupt: 

In [ ]:
nearest_neighbors_auis = umls2020AB_df['lexlm_2000-NN_auis']

nearest_neighbors_strings = []

for nn_auis in tqdm(nearest_neighbors_auis):
    nn_strings = [aui2str[aui] for aui in nn_auis]
    
    nearest_neighbors_strings.append(nn_strings)
    
umls2020AB_df['lexlm_2000-NN_strings'] = nearest_neighbors_strings

In [46]:
recall_cols = umls2020AB_df.filter(regex='.*source_syn_cui_recall').columns

In [47]:
for recall_col in recall_cols:
    atn_recall = []
    for i,row in tqdm(umls2020AB_df.iterrows()):
        recalls = row[recall_col]

        if len(recalls) > 0:
            atn_recall.append(recalls)
        else:
            atn_recall.append([None for i in [0,1,5,10,50,100,200,500,1000,2000]])
        
    recall_col_name = recall_col.split('_recall')[0]
    
    for index,n in tqdm(enumerate([0,1,5,10,50,100,200,500,1000,2000])): 
        umls2020AB_df['R@{}_{}'.format(n,recall_col_name)] = np.array(atn_recall)[:,index] 


0it [00:00, ?it/s]
1it [00:01,  1.51s/it]
1472it [00:01, 1269.39it/s]
2944it [00:01, 2744.85it/s]
4433it [00:01, 4390.08it/s]
5925it [00:01, 6097.12it/s]
7423it [00:02, 7767.36it/s]
8936it [00:02, 9325.58it/s]
10422it [00:02, 10609.99it/s]
11923it [00:02, 11705.81it/s]
13438it [00:02, 12609.10it/s]
14933it [00:02, 13246.00it/s]
16447it [00:02, 13775.71it/s]
17942it [00:02, 14095.83it/s]
19436it [00:02, 14324.56it/s]
20950it [00:02, 14562.89it/s]
22449it [00:03, 14666.17it/s]
23964it [00:03, 14809.01it/s]
25467it [00:03, 14873.55it/s]
26974it [00:03, 14930.38it/s]
28494it [00:03, 15008.55it/s]
30003it [00:03, 14973.00it/s]
31511it [00:03, 15004.29it/s]
33023it [00:03, 15037.33it/s]
34530it [00:03, 15001.91it/s]
36060it [00:03, 15089.34it/s]
37576it [00:04, 15082.91it/s]
39086it [00:04, 15001.58it/s]
40602it [00:04, 15048.06it/s]
42108it [00:04, 15032.09it/s]
43624it [00:04, 15069.52it/s]
45132it [00:04, 15006.14it/s]
46633it [00:04, 14949.31it/s]
48155it [00:04, 15028.92it/s]
49659it [

In [48]:
umls2020AB_df.columns

Index(['0', 'strings', 'auis', '2020AA_synonyms', 'synonym_strings',
       'num_syms', 'sapbert_2000-NN_strings', 'sapbert_2000-NN_auis',
       'sapbert_2000-NN_dist', 'sapbert_2000-NN_recall', 'cuis', 'sem_types',
       'sem_groups', 'source_syns', 'source_syns_plus',
       'sapbert_2000-NN_source_syn_recall',
       'sapbert_2000-NN_source_syn_plus_recall', '2020AA_synonyms_cuis',
       'source_syns_cuis', 'sapbert_2000-NN_cuis',
       'sapbert_2000-NN_cui_recall', 'sapbert_2000-NN_source_syn_cui_recall',
       'number_source_syn_cuis', 'number_source_syn_auis',
       'number_source_syn_plus_auis', 'R@0_sapbert_2000-NN_source_syn_cui',
       'R@1_sapbert_2000-NN_source_syn_cui',
       'R@5_sapbert_2000-NN_source_syn_cui',
       'R@10_sapbert_2000-NN_source_syn_cui',
       'R@50_sapbert_2000-NN_source_syn_cui',
       'R@100_sapbert_2000-NN_source_syn_cui',
       'R@200_sapbert_2000-NN_source_syn_cui',
       'R@500_sapbert_2000-NN_source_syn_cui',
       'R@1000_sapbert_

In [49]:
umls2020AB_df['sources'] = [aui2scui[aui].split('|||')[1] for aui in umls2020AB_df.auis]

In [50]:
umls2020AB_df.groupby('sources').count().sort_values('0',ascending=False)

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@0_sapbert_2000-NN_source_syn_cui,R@1_sapbert_2000-NN_source_syn_cui,R@5_sapbert_2000-NN_source_syn_cui,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui
sources,,,,,,,,,,,,,,,,,,,,,
NCBI,160042,160042,160042,160042,160042,160042,160042,160042,160042,160042,...,14288,14288,14288,14288,14288,14288,14288,14288,14288,14288
RXNORM,68695,68695,68695,68695,68695,68695,68695,68695,68695,68695,...,35930,35930,35930,35930,35930,35930,35930,35930,35930,35930
HGNC,54587,54586,54587,54587,54587,54587,54587,54587,54587,54587,...,50868,50868,50868,50868,50868,50868,50868,50868,50868,50868
MTHSPL,23293,23293,23293,23293,23293,23293,23293,23293,23293,23293,...,16502,16502,16502,16502,16502,16502,16502,16502,16502,16502
MEDCIN,22783,22783,22783,22783,22783,22783,22783,22783,22783,22783,...,9168,9168,9168,9168,9168,9168,9168,9168,9168,9168
MSH,15738,15738,15738,15738,15738,15738,15738,15738,15738,15738,...,10282,10282,10282,10282,10282,10282,10282,10282,10282,10282
SNOMEDCT_US,14597,14597,14597,14597,14597,14597,14597,14597,14597,14597,...,4627,4627,4627,4627,4627,4627,4627,4627,4627,4627
NCI,13758,13758,13758,13758,13758,13758,13758,13758,13758,13758,...,3531,3531,3531,3531,3531,3531,3531,3531,3531,3531
LNC,11300,11300,11300,11300,11300,11300,11300,11300,11300,11300,...,1931,1931,1931,1931,1931,1931,1931,1931,1931,1931


In [51]:
len(umls2020AB_df[(umls2020AB_df.number_source_syn_cuis == 0) & (umls2020AB_df.num_syms > 0)]), len(umls2020AB_df[umls2020AB_df['R@0_sapbert_2000-NN_source_syn_cui'] == 0.0])

(28413, 28413)

In [52]:
at_0_errors = umls2020AB_df[umls2020AB_df['R@0_sapbert_2000-NN_source_syn_cui'] == 0.0]
count_df = at_0_errors.groupby('sem_groups').count()[['0']]
count_df['Group Perc'] = count_df['0']/len(at_0_errors)
display(count_df)

,0,Group Perc
sem_groups,,
Activities & Behaviors,14,0.00
Anatomy,116,0.00
Chemicals & Drugs,20903,0.74
Concepts & Ideas,170,0.01
Devices,421,0.01
Disorders,4220,0.15
Genes & Molecular Sequences,1382,0.05
Geographic Areas,3,0.00
Living Beings,238,0.01


In [53]:
at_0_errors.groupby('sources').count().sort_values('0',ascending=False)

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@0_sapbert_2000-NN_source_syn_cui,R@1_sapbert_2000-NN_source_syn_cui,R@5_sapbert_2000-NN_source_syn_cui,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui
sources,,,,,,,,,,,,,,,,,,,,,
MTHSPL,16502,16502,16502,16502,16502,16502,16502,16502,16502,16502,...,16502,16502,16502,16502,16502,16502,16502,16502,16502,16502
OMIM,2982,2982,2982,2982,2982,2982,2982,2982,2982,2982,...,2982,2982,2982,2982,2982,2982,2982,2982,2982,2982
MTH,2512,2512,2512,2512,2512,2512,2512,2512,2512,2512,...,2512,2512,2512,2512,2512,2512,2512,2512,2512,2512
GS,2036,2036,2036,2036,2036,2036,2036,2036,2036,2036,...,2036,2036,2036,2036,2036,2036,2036,2036,2036,2036
HPO,994,994,994,994,994,994,994,994,994,994,...,994,994,994,994,994,994,994,994,994,994
MDR,746,746,746,746,746,746,746,746,746,746,...,746,746,746,746,746,746,746,746,746,746
MMX,632,632,632,632,632,632,632,632,632,632,...,632,632,632,632,632,632,632,632,632,632
MMSL,575,575,575,575,575,575,575,575,575,575,...,575,575,575,575,575,575,575,575,575,575
GO,489,489,489,489,489,489,489,489,489,489,...,489,489,489,489,489,489,489,489,489,489


In [54]:
umls2020AB_df.filter(regex='.*auis').columns

Index(['auis', 'sapbert_2000-NN_auis', 'number_source_syn_auis',
       'number_source_syn_plus_auis'],
      dtype='object')

In [55]:
pairs_per_row = []

for candidates, sapbert_aui_nn, string, aui, cui in tqdm(zip(all_candidates,umls2020AB_df['sapbert_2000-NN_auis'], umls2020AB_df.strings, umls2020AB_df.auis, umls2020AB_df.cuis)):
    
    true_syn, source_plus_top, labels = candidates
    
    preferred_cui_string = cui2preferred[cui]
    
    num_source = 2000 - len(source_plus_top)
    
    topk = []
    topk_labs = []
    topk_auis = []
    
    i = 0
    for c, l in zip(source_plus_top, labels):
        if i >= num_source:
            topk_auis.append(sapbert_aui_nn[i-num_source])
        else:
            topk_auis.append('')

        if c not in topk:
            topk.append(c)
            topk_labs.append(l)
        
        if len(topk) == 10:
            break
            
        i += 1
        
    
    pref_strings = [cui2preferred[c] for c in topk]
    
    pairs_per_row.append((string, aui, preferred_cui_string, pref_strings, cui, topk, topk_labs, topk_auis))


0it [00:00, ?it/s]
2940it [00:00, 29101.75it/s]
5851it [00:00, 25458.05it/s]
8426it [00:00, 23754.92it/s]
10820it [00:00, 21989.28it/s]
13036it [00:00, 21381.17it/s]
15182it [00:00, 20718.57it/s]
17257it [00:00, 19622.03it/s]
19225it [00:00, 17842.56it/s]
21031it [00:01, 17612.83it/s]
22948it [00:01, 18041.56it/s]
24766it [00:01, 16854.47it/s]
26551it [00:01, 17126.14it/s]
28280it [00:01, 15374.81it/s]
29855it [00:01, 14507.85it/s]
31496it [00:01, 14953.94it/s]
33339it [00:01, 15895.33it/s]
35169it [00:01, 16559.60it/s]
36850it [00:02, 14766.01it/s]
38753it [00:02, 15898.14it/s]
40393it [00:02, 15029.99it/s]
42043it [00:02, 15424.76it/s]
43621it [00:02, 15521.02it/s]
45794it [00:02, 17282.57it/s]
47549it [00:02, 15942.48it/s]
49180it [00:02, 15934.56it/s]
51419it [00:02, 17743.08it/s]
53986it [00:03, 20008.34it/s]
56020it [00:03, 18057.38it/s]
57882it [00:03, 17164.30it/s]
59956it [00:03, 18114.24it/s]
62217it [00:03, 19354.82it/s]
64828it [00:03, 21266.35it/s]
66994it [00:03, 20231.8

In [56]:
umls2020AB_df['top10_preferred_strings'] = pairs_per_row 

In [64]:
num_cuis = []

for string in umls2020AB_df.strings:
    cuis = str2cuis[str(string)]
    num_cuis.append(len(cuis))

umls2020AB_df['num_cuis_per_query_string'] = num_cuis

In [69]:
umls2020AB_df.groupby('num_cuis_per_query_string').count()/len(umls2020AB_df)

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@5_sapbert_2000-NN_source_syn_cui,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings
num_cuis_per_query_string,,,,,,,,,,,,,,,,,,,,,
1,0.97,0.97,0.97,0.97,0.97,0.97,0.97,0.97,0.97,0.97,...,0.37,0.37,0.37,0.37,0.37,0.37,0.37,0.37,0.97,0.97
2,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
3,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [70]:
at_0_errors = umls2020AB_df[umls2020AB_df['R@50_sapbert_2000-NN_source_syn_cui'] == 0.0]

In [71]:
at_0_errors.groupby('num_cuis_per_query_string').count()/len(at_0_errors)

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@5_sapbert_2000-NN_source_syn_cui,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings
num_cuis_per_query_string,,,,,,,,,,,,,,,,,,,,,
1,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,...,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90
2,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
3,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
4,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [58]:
for i,g in at_0_errors.groupby('sem_groups'):
    print('Semantic Group: {}'.format(i))
    print('=' *80)
    print('=' *80)
    print('\n')

    sample_df = g.sample(min(5,len(g)), random_state = np.random.RandomState(42))
    for sample in sample_df.top10_preferred_strings:

        print('Query: {} ||| {}'.format(sample[0], aui2scui[sample[1]].split('|||')[1]))
        print('Correct CUI: {} ||| {}'.format(sample[2], sample[4]))
        print()
        for i,can in enumerate(zip(sample[3],sample[5],sample[6], sample[7])):
            can_s, can_cui, lab, aui = can
            print('Candidate {}: {} ||| {} ||| {} ||| {}'.format(i,can_s, can_cui, aui2str.get(aui, ''), lab))
        print('\n\n' + '='*80 + '\n\n')

Semantic Group: Activities & Behaviors


Query: Friendly behavior ||| OMIM
Correct CUI: Friendly behavior ||| C0679182

Candidate 0: Notable friendly behavior ||| C2749647 ||| Notable friendly behavior ||| False
Candidate 1: Friendly ||| C2700214 ||| Friendly ||| False
Candidate 2: Friendly personality ||| C2749816 ||| Friendly ||| False
Candidate 3: Over-friendly behavior ||| C4747897 ||| Friendly personality ||| False
Candidate 4: Friendly, sociable personality ||| C1844566 ||| Over-friendly behavior ||| False
Candidate 5: Medical Team was Friendly and Kind ||| C4553857 ||| Friendly, sociable personality ||| False
Candidate 6: Office Staff was Friendly and Kind ||| C4553856 ||| Were friendly and kind ||| False
Candidate 7: Overly friendly personality ||| C4746757 ||| Were friendly and kind ||| False
Candidate 8: Friendly behavior ||| C0679182 ||| Overly friendly personality ||| True
Candidate 9: Prosocial Behavior ||| C0871164 ||| friendliness ||| False




Semantic Group: Anatomy




In [72]:
preferred_term_df = pd.DataFrame(cui2preferred.items())

In [74]:
preferred_term_df.groupby(1).count().sort_values(0,ascending=False)

,0
1,
"""Call"" - postponed",1
Retroviral Vector,1
"Retroverted and incarcerated gravid uterus, antepartum",1
"Retroverted and incarcerated gravid uterus, delivered, with mention of postpartum complication",1
"Retroverted and incarcerated gravid uterus, postpartum",1
...,...
Goodyera bilamellata,1
Goodyera bomiensis,1
Goodyera brachyceras,1


In [77]:
umls2020AB_df[umls2020AB_df['cuis'] == 'C1612374']

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings,num_cuis_per_query_string
8617538,34,ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClean...,A32277078,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [SannyTiz...,"[A31248802, A29977087, A29969821, A29969820, A...","[24.19786, 24.682053, 24.682053, 25.122131, 25...","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,(ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClea...,1
8617658,34,ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClean...,A32274751,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [SannyTiz...,"[A31248802, A29977087, A29969821, A29969820, A...","[24.19786, 24.682053, 24.682053, 25.122131, 25...","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,(ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClea...,1
8617480,34,ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClean...,A32278789,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [SannyTiz...,"[A31248802, A29977087, A29969821, A29969820, A...","[24.19786, 24.682053, 24.682053, 25.122131, 25...","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,(ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClea...,1
8617822,33,ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClean...,A32337391,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [Tork Han...,"[A31463247, A19300419, A31248802, A28620165, A...","[19.899017, 20.056366, 20.17865, 20.598282, 20...","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,(ALCOHOL 70 mL in 100 mL TOPICAL GEL [SaniClea...,1
8617852,32,ALCOHOL 2649.787 mL in 3785.41 mL CUTANEOUS GE...,A32331709,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 331.1 mL in 473 mL TOPICAL GEL [Antim...,"[A31559354, A30929839, A18538111, A31561003, A...","[57.674576, 58.09012, 58.592194, 58.893677, 58...","[0.003424657534246575, 0.010273972602739725, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,(ALCOHOL 2649.787 mL in 3785.41 mL CUTANEOUS G...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8616001,10,ALCOHOL 70 mL in 100 mL TOPICAL GEL,A32088239,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [ONE STEP...,"[A31562137, A24304804, A30280240, A31201506, A...","[2.827118, 3.7700806, 3.7700806, 3.7700806, 6....","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,"(ALCOHOL 70 mL in 100 mL TOPICAL GEL, A3208823...",2
8616002,10,ALCOHOL 70 mL in 100 mL TOPICAL GEL,A32088970,"[A28245165, A28678830, A20065575, A20049154, A...",[Ethanol 70% Topical application Gel/Jelly [EP...,292,[ALCOHOL 70 mL in 100 mL TOPICAL GEL [ONE STEP...,"[A31562137, A24304804, A30280240, A31201506, A...","[2.827118, 3.7700806, 3.7700806, 3.7700806, 6....","[0.003424657534246575, 0.017123287671232876, 0...",...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,MTHSPL,"(ALCOHOL 70 mL in 100 mL TOPICAL GEL, A3208897...",2
8615960,10,ALCOHOL 70 mL in 100 mL TOPICAL GEL,A31746729,"[A28245165, A28678830, A20065575, A20049154,

In [84]:
pickle.dump(umls2020AB_df, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_SAPBERT_Source_Info.p','wb'))

In [82]:
#Stratified Split using CUIs to avoid data leakage to test set

umls2020AB_cui_num_syms_df = umls2020AB_df[['cuis','num_syms']].drop_duplicates()

In [85]:
umls2020AB_cui_num_syms_df['no_syms'] = [n == 0 for n in umls2020AB_cui_num_syms_df.num_syms]

In [96]:
training = []
validation = []
testing = []

val = 0.10
test = 0.20

for i,g in umls2020AB_cui_num_syms_df.groupby('no_syms'):
    
    perm_g = g.sample(len(g),random_state=np.random.RandomState(42)).cuis.values
    
    training.extend(perm_g[:len(g) - int(len(g)*(val + test))])
    validation.extend(perm_g[len(g) - int(len(g)*(val + test)):len(g) - int(len(g)*(test))])
    testing.extend(perm_g[len(g) - int(len(g)*test):])
    
    assert(training[-1] != validation[0])
    assert(validation[-1] != testing[0])        

In [98]:
len(training), len(validation), len(testing)

(204012, 29144, 58288)

In [100]:
training = set(training)
validation = set(validation)
testing = set(testing)

In [101]:
split = []

for cui in umls2020AB_df.cuis:
     
    if cui in training:
        split.append('train')
    elif cui in validation:
        split.append('val')
    elif cui in testing:
        split.append('test')

In [102]:
umls2020AB_df['split'] = split

In [108]:
umls2020AB_df.groupby('split').count()

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings,num_cuis_per_query_string
split,,,,,,,,,,,,,,,,,,,,,
test,84912,84912,84912,84912,84912,84912,84912,84912,84912,84912,...,32700,32700,32700,32700,32700,32700,32700,84912,84912,84912
train,303361,303359,303361,303361,303361,303361,303361,303361,303361,303361,...,121327,121327,121327,121327,121327,121327,121327,303361,303361,303361
val,41862,41862,41862,41862,41862,41862,41862,41862,41862,41862,...,16050,16050,16050,16050,16050,16050,16050,41862,41862,41862


In [107]:
dedup_df = []

for i,g in tqdm(umls2020AB_df.groupby(['strings','cuis'])):

    for j, row in g.iterrows():
        dedup_df.append(row)
        break

dedup_df = pd.DataFrame(dedup_df)


 11%|█████████▍                                                                               | 43898/414998 [00:15<01:37, 3793.96it/s]


 22%|███████████████████▌                                                                     | 91100/414998 [03:39<01:20, 4001.05it/s]


 33%|█████████████████████████████▎                                                          | 138013/414998 [03:51<01:11, 3883.46it/s]


 44%|██████████████████████████████████████▉                                                 | 183751/414998 [04:03<00:58, 3929.19it/s]


 55%|████████████████████████████████████████████████▋                                       | 229708/414998 [04:14<00:48, 3823.48it/s]


 66%|██████████████████████████████████████████████████████████▍                             | 275547/414998 [04:26<00:35, 3974.97it/s]


 78%|████████████████████████████████████████████████████████████████████▍                   | 322657/414998 [04:38<00:22, 4037.14it/s]


 89%|██████████████████████████████████████████████████████████████████████████████▏         | 368824/414998 [04:50<00:11, 3902.07it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████▉| 414996/414998 [05:02<00:00, 1374.00it/s]


In [109]:
dedup_df.groupby('split').count()

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings,num_cuis_per_query_string
split,,,,,,,,,,,,,,,,,,,,,
test,82291,82291,82291,82291,82291,82291,82291,82291,82291,82291,...,31589,31589,31589,31589,31589,31589,31589,82291,82291,82291
train,291896,291896,291896,291896,291896,291896,291896,291896,291896,291896,...,114830,114830,114830,114830,114830,114830,114830,291896,291896,291896
val,40809,40809,40809,40809,40809,40809,40809,40809,40809,40809,...,15627,15627,15627,15627,15627,15627,15627,40809,40809,40809


In [110]:
dedup_df.groupby('split').count()/len(dedup_df)

,0,strings,auis,2020AA_synonyms,synonym_strings,num_syms,sapbert_2000-NN_strings,sapbert_2000-NN_auis,sapbert_2000-NN_dist,sapbert_2000-NN_recall,...,R@10_sapbert_2000-NN_source_syn_cui,R@50_sapbert_2000-NN_source_syn_cui,R@100_sapbert_2000-NN_source_syn_cui,R@200_sapbert_2000-NN_source_syn_cui,R@500_sapbert_2000-NN_source_syn_cui,R@1000_sapbert_2000-NN_source_syn_cui,R@2000_sapbert_2000-NN_source_syn_cui,sources,top10_preferred_strings,num_cuis_per_query_string
split,,,,,,,,,,,,,,,,,,,,,
test,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,...,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.20,0.20,0.20
train,0.70,0.70,0.70,0.70,0.70,0.70,0.70,0.70,0.70,0.70,...,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.70,0.70,0.70
val,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,...,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.10,0.10,0.10


In [111]:
dedup_df.columns

Index(['0', 'strings', 'auis', '2020AA_synonyms', 'synonym_strings',
       'num_syms', 'sapbert_2000-NN_strings', 'sapbert_2000-NN_auis',
       'sapbert_2000-NN_dist', 'sapbert_2000-NN_recall', 'cuis', 'sem_types',
       'sem_groups', 'source_syns', 'source_syns_plus',
       'sapbert_2000-NN_source_syn_recall',
       'sapbert_2000-NN_source_syn_plus_recall', '2020AA_synonyms_cuis',
       'source_syns_cuis', 'sapbert_2000-NN_cuis',
       'sapbert_2000-NN_cui_recall', 'sapbert_2000-NN_source_syn_cui_recall',
       'number_source_syn_cuis', 'number_source_syn_auis',
       'number_source_syn_plus_auis', 'R@0_sapbert_2000-NN_source_syn_cui',
       'R@1_sapbert_2000-NN_source_syn_cui',
       'R@5_sapbert_2000-NN_source_syn_cui',
       'R@10_sapbert_2000-NN_source_syn_cui',
       'R@50_sapbert_2000-NN_source_syn_cui',
       'R@100_sapbert_2000-NN_source_syn_cui',
       'R@200_sapbert_2000-NN_source_syn_cui',
       'R@500_sapbert_2000-NN_source_syn_cui',
       'R@1000_sapbert_

In [119]:
dedup_df_simple = dedup_df[['auis','strings','cuis', '2020AA_synonyms', 'source_syns', 'source_syns_plus','sapbert_2000-NN_auis', 'sapbert_2000-NN_cuis', 'source_syns_cuis','split']]

In [115]:
pickle.dump(dedup_df_simple, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_SAPBERT_Source_Info_Official_Split_Basic.p','wb'))

In [122]:
aui_splits = {}
cui_splits = {}

k = 100

for i, row in tqdm(dedup_df_simple.iterrows(), total=len(dedup_df_simple)):
    
    split = row['split']
    
    aui = row['auis']
    cui = row['cuis']
    syns = row['2020AA_synonyms']
    
    aui_samples = aui_splits.get(split, [])
    cui_samples = cui_splits.get(split, [])
    
    for aui_cand in row['source_syns_plus']:
        aui_sample = (aui, aui_cand, 1)
        cui_sample = (aui, cui, 1)
        
        aui_samples.append(aui_sample)
        cui_samples.append(cui_sample)
    
    for aui_cand, cui_cand in zip(row['sapbert_2000-NN_auis'][:k], row['sapbert_2000-NN_cuis'][:k]):
        if cui == cui_cand in syns:
            label = 1
        else:
            label = 0
            
        aui_sample = (aui, aui_cand, label)
        cui_sample = (aui, cui_cand, label)
        
        aui_samples.append(aui_sample)
        cui_samples.append(cui_sample)
        
    aui_splits[split] = aui_samples
    cui_splits[split] = cui_samples


 11%|█████████▍                                                                               | 44290/414996 [00:12<01:36, 3857.25it/s]


 22%|███████████████████▌                                                                     | 91136/414996 [00:24<01:30, 3587.16it/s]


 33%|█████████████████████████████▎                                                          | 138233/414996 [00:37<01:32, 2991.57it/s]


 45%|███████████████████████████████████████▏                                                | 185063/414996 [00:49<00:56, 4057.63it/s]


 56%|█████████████████████████████████████████████████▏                                      | 232018/414996 [01:01<00:43, 4170.04it/s]


 67%|███████████████████████████████████████████████████████████▎                            | 279940/414996 [01:13<00:32, 4132.05it/s]


 79%|█████████████████████████████████████████████████████████████████████▎                  | 326698/414996 [01:25<00:22, 3949.07it/s]


 90%|███████████████████████████████████████████████████████████████████████████████▎        | 373832/414996 [01:38<00:10, 3970.96it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████| 414996/414996 [01:47<00:00, 3864.07it/s]


In [124]:
pickle.dump(aui_splits,open('/data/Bodenreider_UMLS_DL/Interns/Bernal/aui_pairwise_data_splits.{}.p'.format(k),'wb'))
pickle.dump(cui_splits,open('/data/Bodenreider_UMLS_DL/Interns/Bernal/cui_pairwise_data_splits.{}.p'.format(k),'wb'))

In [ ]:
train = pd.read_csv(train_path, sep='\t', quoting=3)


In [112]:
pickle.dump(dedup_df, open('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_SAPBERT_Source_Info_Official_Split.p','wb'))

In [81]:
len(umls2020AB_df), len(umls2020AB_df[['strings']].drop_duplicates())

(430135, 413578)

In [ ]:
# sapbert_errors = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/sapbert_errors_with_pubmed_candidates.csv')

In [ ]:
# eval(sapbert_errors[pd.notna(sapbert_errors['pubmed_candidates'])].loc[430111,'pubmed_candidates'])

# pubmed_candidates = []

# for cs in tqdm(sapbert_errors.pubmed_candidates):
#     cs_auis = []
    
#     if pd.notna(cs):
#         cs = eval(cs)
        
#         cs = [c[0] for c in cs]
        
#         for c in cs:
#             cs_auis.extend(str2aui.get(c,[]))
            
#     pubmed_candidates.append(cs_auis)

In [ ]:
# umls2020AB_df['pubmed_candidates'] = sapbert_errors['pubmed_candidates']

# #Calculating PubMed Candidate Recall @ 1,5,10,50,100
# recall_array = []
# # closest_dist_true = []
# # closest_dist_false = []

# query_synonym = list(umls2020AB_df['2020AA_synonyms'])
# nearest_neighbors = umls2020AB_df['sapbert_2000-NN_auis']

# for true_syn, top, pubmed in tqdm(zip(query_synonym, nearest_neighbors, pubmed_candidates)):
    
#     true_syn = set(true_syn)
    
#     if len(true_syn) > 0:
#         recalls = []

#         topn = set(top)
#         topn = topn.union(set(pubmed))
        
#         true_pos = topn.intersection(true_syn)

#         recalls.append(len(true_pos)/len(true_syn))

#         recall_array.append(recalls)
#     else:
#         recalls = []

#         recall_array.append(recalls)
        
# umls2020AB_df['pubmed_candidates_recall'] = recall_array

In [ ]:
umls2020AB_df.columns

In [ ]:
pd.set_option('max_colwidth',500)

In [ ]:
string_cols = umls2020AB_df.filter(regex='.*NN_strings').columns

In [ ]:
string_cols

In [ ]:
for col in string_cols:
        
    errors = []
    
    for i, row in tqdm(umls2020AB_df.iterrows(),total=len(umls2020AB_df)):
        syns = set(row['synonym_strings'])
        if len(syns) > 0:      
            pred_syns = set(row[col])
            missed_syns = syns.difference(pred_syns)
            errors.append('|||'.join([str(s) for s in missed_syns]))
        else:
            errors.append([])
            
    umls2020AB_df[col.split('_2000')[0]+'_errors'] = errors

In [ ]:
# umls2020AB_df[['strings','auis','2020AA_synonyms','synonym_strings','sapbert_errors']].to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls_2020AB_SAPBERT_errors.csv')

In [ ]:
# umls2020AB_df[(umls2020AB_df['R@2000_sapbert'] == 1.0) & (umls2020AB_df['R@2000_ubert_mlm'] < 1.0) & (umls2020AB_df['0'] < 15)][['0','strings','num_syms','ubert_mlm_errors','R@2000_sapbert','R@2000_ubert_mlm']]


In [ ]:
# f = open('/data/Bodenreider_UMLS_DL/Interns/Bernal/UMLS2020AB_Full_NN_DataFrame_UpToSAPBERT_UBERT.p','rb')

In [ ]:
# umls2020AB_df = pickle.load(f)

In [ ]:
without_R100_errors = umls2020AB_df[umls2020AB_df['R@100_sapbert'] != 0]
len(without_R100_errors)

In [ ]:
for recall_col in without_R100_errors.filter(regex='.*recall.*').columns:
    print(recall_col)
    recall_array = list(without_R100_errors[recall_col].values)
    display(pd.DataFrame(recall_array).describe())

In [ ]:
for n in [1,5,10,50,100,200,500,1000,2000]:
    recall_at_n_cols = umls2020AB_df.filter(regex='R@{}_.*'.format(n)).columns
    umls2020AB_df['R@{}_oracle'.format(n)] = umls2020AB_df[recall_at_n_cols].max(axis=1)

In [ ]:
100*umls2020AB_df.filter(regex='R@.*_oracle').describe()

In [ ]:
nn_aui_cols = umls2020AB_df.filter(regex='.*NN_auis').columns

In [ ]:
import itertools

subsets = []

for L in range(2, len(nn_aui_cols)+1):
    for subset in itertools.combinations(nn_aui_cols, L):
        subsets.append(subset)

In [ ]:
for ss_i, nn_aui_cols_subset in enumerate(subsets):
    recall_array = []

    for i, row in tqdm(umls2020AB_df.iterrows(),total=len(umls2020AB_df)):

        true_syn = row['2020AA_synonyms']
        true_syn = set(true_syn)

        if len(true_syn) > 0:
            recalls = []

            for n in [1,5,10,50,100,200,500,1000,2000]:

                topn = []

                for nn_auis in nn_aui_cols_subset:
                    model_topn = row[nn_auis][:n]
                    topn.extend(model_topn)

                topn = set(topn)
                true_pos = topn.intersection(true_syn)

                recalls.append(len(true_pos)/len(true_syn))

            recall_array.append(recalls)
        else:
            recalls = []

            recall_array.append(recalls)

    umls2020AB_df['ensemble_comb_{}-NN_recall'.format(ss_i)] = recall_array

In [ ]:
[(i,subset) for i, subset in enumerate(subsets)]

In [ ]:
indices = []
dfs = []
i = 0

for recall_col in umls2020AB_df.filter(regex='.*recall.*').columns:
    print(recall_col)
    if 'ensemble' in recall_col:
        indices.append('||'.join(subsets[i]))
        i+=1
    else:
        indices.append(recall_col)
        
    recall_array = list(umls2020AB_df[recall_col].values)
    dfs.append(pd.DataFrame(recall_array).agg(['mean']))

In [ ]:
ensemble = pd.concat(dfs)
ensemble.index = indices

In [ ]:
ensemble.sort_values(4,ascending=False)

In [ ]:
sem_group_analysis = []
sgs = []
num_auis = []

for sg, g in umls2020AB_df.groupby('sem_groups'):
    sem_group_analysis.append(g.filter(regex='R@.*').mean())
    sgs.append(sg)
    num_auis.append(len(g))
    
sem_group_analysis = pd.DataFrame(sem_group_analysis)    
sem_group_analysis['SemGroups'] = sgs
sem_group_analysis['NumAuis'] = num_auis

In [ ]:
sem_group_analysis.filter(regex='SemGroups|NumAuis|.*@100_.*').set_index(['SemGroups','NumAuis'])

In [ ]:
validation_df = []

for sg, g in umls2020AB_df.groupby('sem_groups'):
    validation_df.append(g.sample(int(len(g)*0.2), random_state=np.random.RandomState(42)))

validation_df = pd.concat(validation_df)

In [ ]:
pd.set_option('max_colwidth',500)

In [ ]:
sapbert_correct_ubert_errors = validation_df[(validation_df['R@2000_sapbert'] == 1.0) & (validation_df['R@2000_sapbert_ubert_1392096'] < 1.0)]


In [ ]:
display_sample = sapbert_correct_ubert_errors[:100]

In [ ]:
validation_df

In [ ]:
ubert_correct = validation_df[(validation_df['num_syms'] > 3) & (validation_df['R@2000_sapbert_ubert_1392096'] > 0.5)]
display_sample = ubert_correct[:100]

# display_sample['ubert_mlm_candidate_list_end'] = [[(s,d) for s,d in zip(ss[-10:],ds[-10:])] for ss, ds in zip(display_sample['ubert_mlm_2000-NN_strings'], display_sample['ubert_mlm_2000-NN_dist'])]
display_sample['pubmedbert_candidate_list_start'] = [[(s,d) for s,d in zip(ss[:50],ds[:50])] for ss, ds in zip(display_sample['pubmedbert_2000-NN_strings'], display_sample['pubmedbert_2000-NN_dist'])]
display_sample['pubmedbert_candidate_list_end'] = [[(s,d) for s,d in zip(ss[-50:],ds[-50:])] for ss, ds in zip(display_sample['pubmedbert_2000-NN_strings'], display_sample['pubmedbert_2000-NN_dist'])]

display_sample['sapbert_ubert_candidate_list_start'] = [[(s,d) for s,d in zip(ss[:50],ds[:50])] for ss, ds in zip(display_sample['sapbert_ubert_1392096_2000-NN_strings'], display_sample['sapbert_ubert_1392096_2000-NN_dist'])]
display_sample['sapbert_ubert_candidate_list_end'] = [[(s,d) for s,d in zip(ss[-50:],ds[-50:])] for ss, ds in zip(display_sample['sapbert_ubert_1392096_2000-NN_strings'], display_sample['sapbert_ubert_1392096_2000-NN_dist'])]

In [ ]:
for col in display_sample.filter(regex='.*list_.*').columns:
    
    new_col = []
    
    for row in display_sample[col]:
        
        new_row = []
        
        for elem in row:
            if elem not in new_row:
                new_row.append(elem)
            if len(new_row) > 10:
                break
                
        new_col.append(new_row)
        
    display_sample[col] = new_col

In [ ]:
display_sample.filter(regex='^strings$|.*list_.*|.*sapbert_ubert.*errors')

In [ ]:
display_sample = sapbert_correct_ubert_errors[:100]

# display_sample['ubert_mlm_candidate_list_end'] = [[(s,d) for s,d in zip(ss[-10:],ds[-10:])] for ss, ds in zip(display_sample['ubert_mlm_2000-NN_strings'], display_sample['ubert_mlm_2000-NN_dist'])]
display_sample['sapbert_ubert_candidate_list_start'] = [[(s,d) for s,d in zip(ss[:10],ds[:10])] for ss, ds in zip(display_sample['sapbert_ubert_1392096_2000-NN_strings'], display_sample['sapbert_ubert_1392096_2000-NN_dist'])]
display_sample['sapbert_ubert_candidate_list_end'] = [[(s,d) for s,d in zip(ss[-10:],ds[-10:])] for ss, ds in zip(display_sample['sapbert_ubert_1392096_2000-NN_strings'], display_sample['sapbert_ubert_1392096_2000-NN_dist'])]

display_sample.filter(regex='^strings$|.*sapbert_ubert.*list_.*|.*sapbert_ubert.*errors')

In [ ]:
for recall_col in validation_df.filter(regex='.*recall.*').columns:
    print(recall_col)
    recall_array = list(umls2020AB_df[recall_col].values)
    display(pd.DataFrame(recall_array).describe())

In [ ]:
pd.set_option('max_colwidth',2000)
pd.set_option('max_rows',2000)

In [ ]:
umls2020AB_df['top10'] = [' || '.join([s + '   ' + str(d) + '   '+ str(a) for s,d,a in zip(ss[:10],ds[:10],auis[:10])]) for ss,ds,auis in zip(umls2020AB_df['lexlm_2000-NN_strings'], umls2020AB_df['lexlm_2000-NN_dist'], umls2020AB_df['lexlm_2000-NN_auis'])]

In [ ]:
umls2020AB_df.filter(regex='^auis$|^strings$|top10|num_syms')

In [ ]:
sanity_check = validation_df.sample(200, random_state=np.random.RandomState(42))

In [ ]:
sanity_check['top10'] = [' || '.join([s + '   ' + str(d) + '   '+ str(a) for s,d,a in zip(ss[:10],ds[:10],auis[:10])]) for ss,ds,auis in zip(sanity_check['lexlm_2000-NN_strings'], sanity_check['lexlm_2000-NN_dist'], sanity_check['lexlm_2000-NN_auis'])]
sanity_check.filter(regex='^auis$|^strings$|top10|num_syms')[:10]

In [ ]:
sanity_check['top10'] = [' || '.join([s + '   ' + str(d) + '   '+ str(a) for s,d,a in zip(ss[:10],ds[:10],auis[:10])]) for ss,ds,auis in zip(sanity_check['ubert_mlm_2000-NN_strings'], sanity_check['ubert_mlm_2000-NN_dist'], sanity_check['ubert_mlm_2000-NN_auis'])]
sanity_check.filter(regex='^auis$|^strings$|top10|num_syms')

In [ ]:
sanity_check['top10'] = [' || '.join([s + '   ' + str(d) + '   '+ str(a) for s,d,a in zip(ss[:10],ds[:10],auis[:10])]) for ss,ds,auis in zip(sanity_check['sapbert_400-NN_strings'], sanity_check['sapbert_400-NN_dist'], sanity_check['sapbert_400-NN_auis'])]
sanity_check.filter(regex='^auis$|^strings$|top10|num_syms')

In [ ]:
sanity_check['top10'] = [' || '.join([s + '   ' + str(d) for s,d in zip(ss[:10],ds[:10])]) for ss,ds in zip(sanity_check['pubmedbert_2000-NN_strings'], sanity_check['pubmedbert_2000-NN_dist'])]
sanity_check.filter(regex='^auis$|^strings$|top10|num_syms')

In [ ]:
sanity_check['top10'] = [' || '.join([s + '   ' + str(d) for s,d in zip(ss[:10],ds[:10])]) for ss,ds in zip(sanity_check['krissbert_2000-NN_strings'], sanity_check['krissbert_2000-NN_dist'])]
sanity_check.filter(regex='^auis$|^strings$|top10|num_syms')

In [ ]:
validation_df['top10'] = [s[:10] for s in validation_df['sapbert_400-NN_strings']]
validation_df['top10_auis'] = [s[:10] for s in validation_df['sapbert_400-NN_auis']]
validation_df['top10_dist'] = [s[:10] for s in validation_df['sapbert_400-NN_dist']]
validation_df['top100'] = [s[:100] for s in validation_df['sapbert_400-NN_strings']]
validation_df['top100_dist'] = [s[:100] for s in validation_df['sapbert_400-NN_dist']]

In [ ]:
non_sym_validation = validation_df[validation_df['num_syms'] == 0]
non_sym_samples = []

for i,g in non_sym_validation.groupby('sem_groups'):
    print(i)
    if len(g) > 10:
        non_sym_sample = g.sample(10, random_state=np.random.RandomState(42))
        non_sym_sample = non_sym_sample.filter(regex='^auis$|^strings$|top10$|top10_')
        non_sym_sample['SemGroup'] = i
        non_sym_samples.append(non_sym_sample)

In [ ]:
pd.concat(non_sym_samples)

In [ ]:
errors = validation_df[validation_df['R@100_sapbert'] == 0.0]

In [ ]:
errors.columns

In [ ]:
errors.groupby('sem_groups').count()

In [ ]:
error_samples = []

for i,g in errors.groupby('sem_groups'):
    if len(g) > 80:
        print(i)
        error_sample = g[['auis','strings','2020AA_synonyms','synonym_strings','num_syms','top10','top10_auis']].sample(10, random_state=np.random.RandomState(42))
        error_sample['SemGroup'] = i
        error_samples.append(error_sample)

In [ ]:
pd.concat(error_samples)

In [ ]:
distance_at_100 = []

for i,row in validation_df.iterrows():
    distances = row['sapbert_400-NN_dist']
    
    distance_at_100.append(distances[100])    
    
validation_df['sb_dist@100'] = distance_at_100

distance_100_threshold = validation_df[validation_df['num_syms'] > 0]['sb_dist@100'].mean()

In [ ]:
#Calculating Recall & # of False Positives @ Distance Threshold in Validation Set

nearest_neighbors_auis = umls2020AB_df['sapbert_400-NN_auis']
nearest_neighbors_dist = umls2020AB_df['sapbert_400-NN_dist']

recall_array = []
fps = []

for true_syn, top_auis, top_dist in tqdm(zip(query_synonym_auis, nearest_neighbors_auis, nearest_neighbors_dist)):
    
    if len(true_syn) > 0:
        true_syn = set(true_syn)

        n = len(np.where(top_dist < distance_100_threshold)[0])

        topn = set(top_auis[:n])
        true_pos = topn.intersection(true_syn)

        recall_array.append(len(true_pos)/len(true_syn))
        fps.append(n-len(true_pos))
    else:
        recalls = None

        recall_array.append(recalls)

        n = len(np.where(top_dist < distance_100_threshold)[0])
        fps.append(n)

umls2020AB_df['sb_fps'] = fps
umls2020AB_df[umls2020AB_df['num_syms'] == 0].sb_fps.mean(),umls2020AB_df[umls2020AB_df['num_syms'] > 0].sb_fps.mean()

In [ ]:
pd.DataFrame(recall_array).describe()

In [ ]:
distance_at_100 = []

for i,row in validation_df.iterrows():
    distances = row['lexlm_2000-NN_dist']
    
    distance_at_100.append(distances[100])    
    
validation_df['lm_dist@100'] = distance_at_100

distance_100_threshold = validation_df[validation_df['num_syms'] > 0]['lm_dist@100'].mean()

In [ ]:
#Calculating Recall & # of False Positives @ Distance Threshold in Validation Set

nearest_neighbors_auis = umls2020AB_df['lexlm_2000-NN_auis_x']
nearest_neighbors_dist = umls2020AB_df['lexlm_2000-NN_dist']

recall_array = []
fps = []

for true_syn, top_auis, top_dist in tqdm(zip(query_synonym_auis, nearest_neighbors_auis, nearest_neighbors_dist)):
    
    if len(true_syn) > 0:
        true_syn = set(true_syn)

        n = len(np.where(top_dist < distance_100_threshold)[0])

        topn = set(top_auis[:n])
        true_pos = topn.intersection(true_syn)

        recall_array.append(len(true_pos)/len(true_syn))
        fps.append(n-len(true_pos))
    else:
        recalls = None

        recall_array.append(recalls)

        n = len(np.where(top_dist < distance_100_threshold)[0])
        fps.append(n)

umls2020AB_df['lm_fps'] = fps
umls2020AB_df[umls2020AB_df['num_syms'] == 0].lm_fps.mean(),umls2020AB_df[umls2020AB_df['num_syms'] > 0].lm_fps.mean()

In [ ]:
validation_df.columns

In [ ]:
bins = []
large_bin = []

for i,g in umls2020AB_df.groupby('num_syms'):
    if i < 10:
        bins.append((i, g['R@100_SB'].mean()))
    else:
        large_bin.append(g)
        
bins.append(('10+',pd.concat(large_bin)['R@100_SB'].mean()))

In [ ]:
pd.DataFrame(bins)

In [ ]:
bins = []
large_bin = []

for i,g in umls2020AB_df.groupby('num_syms'):
    if i < 10:
        bins.append((i, g['R@100_LM'].mean()))
    else:
        large_bin.append(g)
        
bins.append(('10+',pd.concat(large_bin)['R@100_LM'].mean()))

In [ ]:
pd.DataFrame(bins)